
## Exercicios Técnicas de Treinamento



In [1]:
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import torch.optim as optim
import torch.nn as nn
import torch

plt.rcParams['figure.figsize'] = (8, 8)

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_size = 28):
      super(NeuralNetwork, self).__init__()
      self.flatten = nn.Flatten()
      self.fc1 = nn.Linear(input_size*input_size, 2048)
      self.fc2 = nn.Linear(2048, 512)
      self.fc3 = nn.Linear(512, 10)


    def forward(self, x):
      x = self.flatten(x)
      x = self.fc1(x)
      x = torch.sigmoid(x)
      x = self.fc2(x)
      x = torch.sigmoid(x)
      x = self.fc3(x)
      return x

In [ ]:
def train(model, device, train_loader, optimizer, criterion):
    model.train()
    total_loss = 0
    
    for data, target in train_loader:
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()

        output = model(data)
        loss = criterion(output, target)
    
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()

    return total_loss

In [ ]:
def test(model, device, test_loader, criterion):
    model.eval()
    test_loss, correct = 0, 0
    
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            
            test_loss += criterion(output, target).item()
            
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    return test_loss, correct / len(test_loader.dataset)

In [ ]:
device = torch.device("cuda:0")
epocas = 10

train_set = datasets.MNIST("../data", train=True, download=True, transform=transforms.ToTensor())
test_set = datasets.MNIST("../data", train=False, transform=transforms.ToTensor())

train_set, val_set = torch.utils.data.random_split(train_set, [0.7,0.3])

size = test_set[0][0].size()[2]

train_loader = torch.utils.data.DataLoader(train_set, batch_size = 64, shuffle = True)
val_loader = torch.utils.data.DataLoader(val_set, batch_size = 1024, shuffle = False)
test_loader = torch.utils.data.DataLoader(test_set, batch_size = 1024, shuffle = False)

model = NeuralNetwork(size).to(device)
criterion = nn.CrossEntropyLoss(reduction = "sum")
optimizer = optim.Adadelta(model.parameters())

val_losses, train_losses, test_losses, epochs = [], [], [], []
for epoch in range(1, epocas + 1):
    print("Epoch:", epoch)
    
    train_loss = train(model, device, train_loader, optimizer, epoch, criterion)
    val_loss, _ = test(model, device, val_loader, criterion)

    val_losses.append(val_loss)
    train_losses.append(train_loss)
    epochs.append(epoch)

    optimizer.step()

_, accuracy = test(model, device, test_loader, criterion)
print("Final model accuracy:", accuracy)

In [ ]:
plt.xlabel("Época")
plt.ylabel("Loss")
plt.plot(epochs, train_losses, '+-', color='blue', label="Train Loss")
plt.plot(epochs, val_losses, '.-', color='black', label="Validation Loss")
plt.legend(loc="upper right")

plt.show()